In [1]:
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
from elasticsearch import Elasticsearch
import json
import os
import pandas as pd

# Create the client instance
client = Elasticsearch("http://localhost:9200")

# Successful response!
client.info()

C:\Users\XDavi\AppData\Local\Temp\ipykernel_17960\2156522912.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.info()


ObjectApiResponse({'name': '01df5d9e4e66', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'FITIbcYeR4erqq6Km1jxLQ', 'version': {'number': '7.17.9', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'ef48222227ee6b9e70e502f0f0daa52435ee634d', 'build_date': '2023-01-31T05:34:43.305517834Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [11]:
# NOTE: THe following code must be run just once
# Get the documents from the directory
entries = os.listdir('../1-publications/json')
for i in entries:
    # Opening JSON file
    f = open('../1-publications/json/'+i)

    # returns JSON object as a dictionary
    data = json.load(f)
    
    # Closing file
    f.close()

    # Remove the unnecessary fields
    list = ['url','paperId','year','publicationDate','authors']
    for i in list:
        data.pop(i, None)
    client.index(
    index='prueba',
    document=data)

# Refresh the index
client.indices.refresh(index='prueba')

C:\Users\XDavi\AppData\Local\Temp\ipykernel_17960\1647646017.py:18: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.index(
C:\Users\XDavi\AppData\Local\Temp\ipykernel_17960\1647646017.py:23: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.indices.refresh(index='prueba')


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [15]:
# Query for the keyword you want
val = input("Enter the keyword you are looking for: ")
result = client.search(
 index='prueba',
  query={
    'match': {'abstract': val}
  }
 )
a = result['hits']['hits']
df = pd.DataFrame(columns=['title', 'abstract','score'])
for i in a:
    df = df.append({'title': i['_source']['title'], 'abstract': i['_source']['abstract'],'score':i['_score']}, ignore_index=True)
df

C:\Users\XDavi\AppData\Local\Temp\ipykernel_17960\2039512892.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = client.search(
C:\Users\XDavi\AppData\Local\Temp\ipykernel_17960\2039512892.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': i['_source']['title'], 'abstract': i['_source']['abstract'],'score':i['_score']}, ignore_index=True)


,title,abstract,score
0,"Comparison between Expert Systems, Machine Lea...","Today, the science of artificial intelligence ...",2.165986


In [10]:
# DO NOT RUN THIS CODE IF YOU DON'T WANT TO DELETE THE INDEX!!!
#client.options(ignore_status=[400,404]).indices.delete(index='prueba')

C:\Users\XDavi\AppData\Local\Temp\ipykernel_17960\986347348.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.options(ignore_status=[400,404]).indices.delete(index='prueba')


ObjectApiResponse({'acknowledged': True})